# АНАЛИЗ УНИКАЛЬНЫХ ПОЛЬЗОВАТЕЛЬСКИХ ПУТЕЙ

In [146]:
# Таблица Event = a
# id	идентификатор события
# user_id	уникальный идентификатор пользователя, совершившего событие в приложении
# start_time	дата и время события
# event_type	тип события (значения: registration — регистрация; tutorial_start — начало обучения; tutorial_finish — завершение обучения;/ 
# /level_choice — выбор уровня сложности; pack_choice — выбор пакетов вопросов)
# tutorial_id	идентификатор обучения (этот идентификатор есть только у событий обучения)
# selected_level	выбранный уровень сложности обучения

# Таблица purchase = b
# id	идентификатор события
# user_id	уникальный идентификатор пользователя, совершившего событие в приложении
# event_datetime	дата и время события/покупки
# amount	сумма оплаты

In [147]:
import pandas as pd

a=pd.read_csv('E:/Загрузки/файлы для анализа/1/7_4_Events (2).csv', sep = ',')
b=pd.read_csv('E:/Загрузки/файлы для анализа/1/purchase1.csv', sep = ',')
a1=a.copy()

# знакомимся с данными
display(a.info())
display(b.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252334 entries, 0 to 252333
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              252334 non-null  int64  
 1   event_type      252334 non-null  object 
 2   selected_level  31086 non-null   object 
 3   start_time      252334 non-null  object 
 4   tutorial_id     125103 non-null  float64
 5   user_id         252334 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 11.6+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5956 entries, 0 to 5955
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              5956 non-null   int64 
 1   user_id         5956 non-null   int64 
 2   event_datetime  5956 non-null   object
 3   amount          5956 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 186.2+ KB


None

# Анализ буду производить по данным о пользователях, зарегистрировавшихся в 2018 году
нюанс: есть пользователи, которые не начинают обучение, есть такие, кто его не заканчивает, а есть те, кто проходят его по несколько раз.

In [148]:
# фильтруем по условию
cond = (a.start_time >= '2018-01-01') & (a.start_time < '2019-01-01') & (a.event_type == 'registration')
registered = a[cond]['user_id'].to_list()
events_2018 = a[a.user_id.isin(registered)].copy()
events_2018['start_time'] = pd.to_datetime( events_2018.start_time, format='%Y-%m-%dT%H:%M:%S')
display(events_2018.info())
display(events_2018.head(30).iloc[::5])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66959 entries, 51405 to 118364
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              66959 non-null  int64         
 1   event_type      66959 non-null  object        
 2   selected_level  8342 non-null   object        
 3   start_time      66959 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         66959 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 3.6+ MB


None

,id,event_type,selected_level,start_time,tutorial_id,user_id
51405,80308,registration,NaN,2018-01-01 03:48:40,NaN,27832
51410,80313,registration,NaN,2018-01-01 14:08:40,NaN,27837
51415,80318,tutorial_start,NaN,2018-01-01 15:40:43,31507.0,27836
51420,80323,tutorial_start,NaN,2018-01-01 19:11:36,31509.0,27839
51425,80328,pack_choice,NaN,2018-01-01 20:38:43,NaN,27835
51430,80333,registration,NaN,2018-01-01 23:18:46,NaN,27842


# Готовим 2 таблицу

In [149]:
b_list = a[cond]['user_id'].to_list()
purchase_2018 = b[b.user_id.isin(b_list)].copy()
purchase_2018['event_datetime'] = pd.to_datetime( purchase_2018.event_datetime, format='%Y-%m-%dT%H:%M:%S')
display(purchase_2018.head())
purchase_2018.info()

,id,user_id,event_datetime,amount
1171,16845,27845,2018-01-03 18:53:43,100
1172,16846,27865,2018-01-04 14:46:10,250
1174,16848,27911,2018-01-07 08:19:12,50
1175,16849,27910,2018-01-07 12:11:34,100
1176,16850,27940,2018-01-07 13:16:41,200


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600 entries, 1171 to 2778
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              1600 non-null   int64         
 1   user_id         1600 non-null   int64         
 2   event_datetime  1600 non-null   datetime64[ns]
 3   amount          1600 non-null   int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 62.5 KB


# Объединим датафреймы

In [150]:
# добавим столбец с типом во 2 таблицу
purchase_2018['event_type'] = 'purchase'
# меняем названия столбцов для удобства (таблицы не связаны по id, это две разные таблицы, единственное совпадения по столбцу user_id )
events_2018 = events_2018.rename(columns={"id": "event_id"})
purchase_2018 = purchase_2018.rename(columns={"id": "purchase_id"})
purchase_2018 = purchase_2018.rename(columns={"event_datetime": "start_time"})
total_events_df = pd.concat([events_2018,purchase_2018],sort=False)
total_events_df.head(10)

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,amount
51405,80308.0,registration,NaN,2018-01-01 03:48:40,NaN,27832,NaN,NaN
51406,80309.0,registration,NaN,2018-01-01 04:07:25,NaN,27833,NaN,NaN
51407,80310.0,registration,NaN,2018-01-01 08:35:10,NaN,27834,NaN,NaN
51408,80311.0,registration,NaN,2018-01-01 11:54:47,NaN,27835,NaN,NaN
51409,80312.0,registration,NaN,2018-01-01 13:28:07,NaN,27836,NaN,NaN
51410,80313.0,registration,NaN,2018-01-01 14:08:40,NaN,27837,NaN,NaN
51411,80314.0,registration,NaN,2018-01-01 14:42:58,NaN,27838,NaN,NaN
51412,80315.0,tutorial_start,NaN,2018-01-01 14:54:40,31505.0,27836,NaN,NaN
51413,80316.0,tutorial_start,NaN,2018-01-01 15:00:51,31506.0,27835,NaN,NaN
51414,80317.0,tutorial_finish,NaN,2018-01-01 15:06:15,31506.0,27835,NaN,NaN


In [151]:
total_events_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68559 entries, 51405 to 2778
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_id        66959 non-null  float64       
 1   event_type      68559 non-null  object        
 2   selected_level  8342 non-null   object        
 3   start_time      68559 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         68559 non-null  int64         
 6   purchase_id     1600 non-null   float64       
 7   amount          1600 non-null   float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(2)
memory usage: 4.7+ MB


# Теперь сбросим индексы объединённого датафрейма (так как после объединения они дублировались и несут мало смысла)

In [152]:
total_events_df = total_events_df.reset_index(drop=True).sort_values(['user_id','start_time'])
total_events_df

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,amount
0,80308.0,registration,NaN,2018-01-01 03:48:40,NaN,27832,NaN,NaN
1,80309.0,registration,NaN,2018-01-01 04:07:25,NaN,27833,NaN,NaN
12,80320.0,tutorial_start,NaN,2018-01-01 17:47:40,31508.0,27833,NaN,NaN
13,80321.0,tutorial_finish,NaN,2018-01-01 17:50:08,31508.0,27833,NaN,NaN
2,80310.0,registration,NaN,2018-01-01 08:35:10,NaN,27834,NaN,NaN
...,...,...,...,...,...,...,...,...
66954,147262.0,tutorial_start,NaN,2019-01-01 03:26:18,49554.0,47756,NaN,NaN
66955,147263.0,tutorial_finish,NaN,2019-01-01 03:29:25,49554.0,47756,NaN,NaN
66957,147266.0,level_choice,medium,2019-01-01 05:42:11,NaN,47756,NaN,NaN
66958,147267.0,pack_choice,NaN,2019-01-01 05:50:36,NaN,47756,NaN,NaN


# Воссоздаём последовательность событий

In [153]:
user_path_df = total_events_df.groupby(["user_id"])["event_type"].apply(list).reset_index()
user_path_df.head(10)

,user_id,event_type
0,27832,[registration]
1,27833,"[registration, tutorial_start, tutorial_finish]"
2,27834,"[registration, tutorial_start, tutorial_finish]"
3,27835,"[registration, tutorial_start, tutorial_finish..."
4,27836,"[registration, tutorial_start, tutorial_start,..."
5,27837,[registration]
6,27838,[registration]
7,27839,"[registration, tutorial_start, tutorial_finish..."
8,27840,"[registration, tutorial_start, level_choice]"
9,27841,"[registration, tutorial_start, tutorial_finish]"


#  Выявляем наиболее популярные пути

In [154]:
# преобразуем список в строку в каждой ячейке, это ускорит обработку данных
user_path_df["event_path"] = user_path_df["event_type"].apply(lambda x: " > ".join(x))
user_path_df["event_path"].head(10) 

KeyError: 'event_path'

In [ ]:
user_paths = (
    user_path_df.groupby(["event_path"])["user_id"]
    .nunique()
    .sort_values(ascending=False)
)
user_paths.head(10)

event_path
registration                                                                                                       7970
registration > tutorial_start > tutorial_finish > level_choice > pack_choice                                       2796
registration > tutorial_start > tutorial_finish                                                                    1956
registration > tutorial_start > tutorial_finish > level_choice                                                     1713
registration > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                            1083
registration > tutorial_start                                                                                       842
registration > tutorial_start > level_choice > pack_choice                                                          346
registration > tutorial_start > tutorial_finish > tutorial_start > tutorial_finish                                  323
registration > tutorial_start

Самый популярный путь, который приводит к оплате:
registration > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase 

# Ищем последовательности с оплатой

In [ ]:
user_paths[user_paths.index.str.contains('purchase')].head(10)

event_path
registration > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                                                                                                             1083
registration > tutorial_start > level_choice > pack_choice > purchase                                                                                                                                124
registration > tutorial_start > tutorial_finish > level_choice > pack_choice > tutorial_start > tutorial_finish > purchase                                                                           101
registration > tutorial_start > tutorial_finish > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                                                                            52
registration > tutorial_start > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                                                                                 

Большинство последовательностей, которые содержат в себе оплату, также содержат старт обучения. Это наводит меня на гипотезу, что вероятность оплаты зависит от того, проходил ли пользователь обучение.